# LU Decomposition Efficiency Comparison

Compare efficiency of LU decomposition vs repeated Gaussian elimination for solving multiple linear systems with the same coefficient matrix.

In [ ]:
import time
import random
from src.matrix import (
    Matrix, gaussian_elimination, plu_decomposition, solve_lu
)

## Helper Functions

Generate random matrices and vectors for benchmarking.

In [ ]:
def generate_random_matrix(n: int) -> Matrix:
    """Generate random n x n matrix."""
    data = [[random.uniform(-10, 10) for _ in range(n)] for _ in range(n)]
    return Matrix(data)


def generate_random_vector(n: int) -> list[float]:
    """Generate random vector of length n."""
    return [random.uniform(-10, 10) for _ in range(n)]

## Benchmark Functions

Time both approaches for solving multiple systems.

In [ ]:
def benchmark_gaussian(A: Matrix, bs: list[list[float]]) -> float:
    """Time solving multiple systems with Gaussian elimination."""
    start = time.perf_counter()
    for b in bs:
        try:
            gaussian_elimination(A, b)
        except ValueError:
            pass  # Singular matrix
    return time.perf_counter() - start


def benchmark_lu(A: Matrix, bs: list[list[float]]) -> float:
    """Time solving multiple systems with LU decomposition."""
    start = time.perf_counter()
    try:
        P, L, U = plu_decomposition(A)
        for b in bs:
            solve_lu(L, U, b, P)
    except ValueError:
        pass  # Singular matrix
    return time.perf_counter() - start

## Run Benchmarks

Compare performance across different matrix sizes and number of systems.

In [ ]:
def run_benchmark():
    """Run benchmarks for various matrix sizes and number of systems."""
    print("Benchmarking Gaussian vs LU decomposition")
    print("=" * 60)

    matrix_sizes = [10, 20, 50, 100]
    num_systems = [1, 5, 10, 20, 50]

    for n in matrix_sizes:
        print(f"\nMatrix size: {n} x {n}")
        print("-" * 40)

        A = generate_random_matrix(n)

        for k in num_systems:
            bs = [generate_random_vector(n) for _ in range(k)]

            t_gauss = benchmark_gaussian(A, bs)
            t_lu = benchmark_lu(A, bs)

            speedup = t_gauss / t_lu if t_lu > 0 else float('inf')

            print(f"  {k:3d} systems: Gauss={t_gauss:.4f}s, LU={t_lu:.4f}s, Speedup={speedup:.2f}x")


# Run the benchmark
random.seed(42)  # For reproducibility
run_benchmark()

## Theoretical Complexity Analysis

Understanding why LU is more efficient for multiple systems.

In [ ]:
def theoretical_complexity():
    """
    Explain theoretical complexity.

    Gaussian Elimination for k systems:
        k * O(n^3) = O(kn^3)

    LU Decomposition for k systems:
        O(n^3) + k * O(n^2) = O(n^3 + kn^2)

    LU wins when k is large!

    Break-even point:
        kn^3 = n^3 + kn^2
        k(n^3 - n^2) = n^3
        k = n^3 / (n^3 - n^2) = n / (n - 1)

    For n = 10: k = 1.1
    For n = 100: k = 1.01

    So LU is almost always better for k >= 2!
    """
    print("Theoretical Analysis")
    print("=" * 60)
    print("Gaussian: O(kn^3) for k systems")
    print("LU:       O(n^3 + kn^2) for k systems")
    print("\nLU wins when k > n/(n-1) = 1 for large n")
    print("\nBreak-even points:")
    for n in [10, 20, 50, 100]:
        k = n / (n - 1)
        print(f"  n = {n:3d}: k = {k:.3f}")
    print("\nConclusion: LU is almost always better for k >= 2!")


theoretical_complexity()

## Visualization

Plot the speedup as a function of number of systems.

In [ ]:
import matplotlib.pyplot as plt

def plot_speedup():
    """Visualize speedup of LU over Gaussian elimination."""
    matrix_sizes = [10, 20, 50]
    num_systems = [1, 2, 5, 10, 20, 50]
    
    plt.figure(figsize=(10, 6))
    
    for n in matrix_sizes:
        speedups = []
        A = generate_random_matrix(n)
        
        for k in num_systems:
            bs = [generate_random_vector(n) for _ in range(k)]
            t_gauss = benchmark_gaussian(A, bs)
            t_lu = benchmark_lu(A, bs)
            speedup = t_gauss / t_lu if t_lu > 0 else 1
            speedups.append(speedup)
        
        plt.plot(num_systems, speedups, marker='o', label=f'{n}x{n} matrix')
    
    plt.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Break-even')
    plt.xlabel('Number of Systems (k)')
    plt.ylabel('Speedup (Gaussian time / LU time)')
    plt.title('LU Decomposition Speedup over Gaussian Elimination')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xscale('log')
    plt.tight_layout()
    plt.show()


random.seed(42)
plot_speedup()

## Key Takeaways

1. **Single system**: Gaussian elimination and LU have similar performance (LU has slight overhead for storing L and U).

2. **Multiple systems**: LU becomes increasingly faster because:
   - Factorization is done once: O(n^3)
   - Each solve is only O(n^2)

3. **Rule of thumb**: Use LU when solving 2+ systems with the same coefficient matrix.

4. **Real-world applications**:
   - Circuit simulation (same network, different inputs)
   - Structural analysis (same structure, different loads)
   - Control systems (same dynamics, different initial conditions)
   - Machine learning (iterative methods with same Hessian)